In [1]:
from bokeh.io import output_notebook
from bokeh.plotting import show, figure
from bokeh.palettes import Colorblind
output_notebook()

import numpy as np
import pandas as pd
import astropy.units as u
import virga.justdoit as jdi
import virga.compare_plots as jpi
import matplotlib.pyplot as plt
import time
from bokeh.plotting import show, figure
from bokeh.layouts import column
from virga.direct_mmr_solver import generate_altitude
import time


Loading BokehJS ...

In [2]:
# locate mieff files
mieff_directory = "~/Documents/codes/all-data/mieff_files"

metallicity = 1 #atmospheric metallicity relative to Solar
mean_molecular_weight = 2.2 # atmospheric mean molecular weight
fsed = 1


a = jdi.Atmosphere(['MnS','Cr','MgSiO3','Fe'], fsed=fsed,mh=metallicity, mmw = mean_molecular_weight)
#,'Cr','MgSiO3','Fe'
# set the planet gravity
grav = 7.460
a.gravity(gravity=grav, gravity_unit=u.Unit('m/(s**2)'))

# generate temperature, pressure and kz profiles
df = jdi.hot_jupiter()
pres = np.array(df["pressure"])
temp = np.array(df["temperature"])
kz = np.array(df["kz"])
gravity = grav*100

In [3]:
mieff_directory = "~/Documents/codes/all-data/mieff_files"
TP_directory = "~/Documents/codes/all-data/Mark_data/"
filenames = ["t1000g1000nc_m0.0.dat", "t1500g1000nc_m0.0.dat", "t1700g1000f3_m0.0k.dat", \
                    "t200g3160nc_m0.0.dat", "t2400g3160nc_m0.0.dat" ]
filename = TP_directory + filenames[0]
    
#   define atmosphere properties
df = pd.read_csv(filename, delim_whitespace=True, usecols=[1,2], header=None)
df.columns = ["pressure", "temperature"]
pres = np.array(df["pressure"])[1:]
temp = np.array(df["temperature"])[1:]
gravity = df["pressure"][0] * 100 
kz = np.zeros(len(pres)) + 1e9

metallicity = 1 #atmospheric metallicity relative to Solar
mean_molecular_weight = 2.2 # atmospheric mean molecular weight
#get pyeddy recommendation for which gases to run
recommended_gases = jdi.recommend_gas(pres, temp,
                                             metallicity, mean_molecular_weight)

a = jdi.Atmosphere([recommended_gases[0]], fsed=fsed, mh=metallicity, mmw=mean_molecular_weight)
a.gravity(gravity=grav, gravity_unit=u.Unit('cm/(s**2)'))

/Users/crooney/Documents/codes/virga/virga/root_functions.py:225: RuntimeWarning: divide by zero encountered in log10
  return np.log10(pv) - np.log10(partial_p)


In [4]:
#   compare solvers: virga unrefined TP, virga refined TP, new solver
# solver = [True, True, False] # true means virga, false is new solver

output = []
pressure_vals = []
for eps in [1e2,1e1,1e0,1e-1]:
    # refine profiles (increase number of pressure values in dataframe)
    z, pres_, P_z, temp_, T_z, T_P, kz_ = generate_altitude(pres, temp, kz, gravity, 
                                                mean_molecular_weight, eps, refine_TP=True)  
    pressure_vals.append(len(pres_))
    a.ptk(df = pd.DataFrame({'pressure':pres_, 'temperature':temp_, 'kz':kz_}))
        
    t1 = time.time()
    all_out = jdi.compute(a, eps, as_dict=True, directory=mieff_directory, layers=True, quick_stop=False)
    t2 = time.time() - t1
    print("time = ", t2)
    output.append(all_out)

print("done")

eddysed time =  0.06783008575439453
eddysed qc_path =  [373.62344276]
time =  0.07747912406921387
eddysed time =  0.17789077758789062
eddysed qc_path =  [373.81172661]
time =  0.18623018264770508
eddysed time =  1.249513864517212
eddysed qc_path =  [373.7583808]
time =  1.2586109638214111
eddysed time =  11.378329992294312
eddysed qc_path =  [373.75356785]
time =  11.387943983078003
done


In [5]:
# show that all temperature-pressure profiles are the same
#labels = ["virga", "virga refined", "new"]
labels = []; lines = []
for i in range(len(pressure_vals)):
    labels.append("length = %d" % pressure_vals[i])
    lines.append("solid")
show(jpi.plot_pt(output, labels, lines, plot_height=450))

In [6]:
# compare mean particle radius rg for each solver (displaying result for MnS)
show(jpi.plot_output(output, "mean_particle_r", "Mean Particle Radius (um)", labels, lines))

In [7]:
# compare column number density (displaying result for MnS)
show(jpi.plot_output(output, "column_density", "Number Density", labels, lines))

In [8]:
# print("Non-refned PT profile: Virga")
for i in range(len(pressure_vals)):
    print(labels[i])
    show(jpi.all_optics(output[i]))

# print("Refned PT profile: Virga")
# show(jpi.all_optics(output[1]))

# print("New solver")
# show(jpi.all_optics(output[2]))

length = 91


KeyError: 'opd_per_layer'

In [ ]:
show(jpi.plot_cumsum(output,labels,lines))

In [ ]:
len(output[0]["altitude"])

In [ ]:
len(output[0]["column_density"])